# 命令式和符号式混合编程

In [ ]:
import d2l
from mxnet import np, npx, sym
from mxnet.gluon import nn
npx.set_np()

def add(a, b):
    return a + b
def fancy_func(a, b, c):
    e = add(a, b)
    return add(c, e)
fancy_func(1, 2, 3)

符号式编程。

In [ ]:
def add_str():
    return '''def add(a, b):
    return a + b
'''
def fancy_func_str():
    return '''def fancy_func(a, b, c):
    e = add(a, b)
    return add(c, e)
'''
def evoke_str():
    return add_str() + fancy_func_str() + '''
print(fancy_func(1, 2, 3))
'''
prog = evoke_str()
y = compile(prog, '', 'exec')
exec(y)

使用 ``HybridSequential`` 类来构建网络。

In [ ]:
def get_net():
    net = nn.HybridSequential()
    net.add(nn.Dense(256, activation='relu'),
            nn.Dense(128, activation='relu'),
            nn.Dense(2))
    net.initialize()
    return net

x = np.random.normal(size=(1, 512))
net = get_net()
net(x)

编译优化并运行。

In [ ]:
net.hybridize()
net(x)

测试速度。

In [ ]:
def benchmark(net, x):
    timer = d2l.Timer()
    for i in range(1000):
        _ = net(x)
    npx.waitall()
    return timer.stop()

net = get_net()
print('before hybridizing: %.4f sec' % (benchmark(net, x)))
net.hybridize()
print('after hybridizing: %.4f sec' % (benchmark(net, x)))

导出到其他语言。

In [ ]:
net.export('my_mlp')
!ls my_mlp*
!head -n20 my_mlp-symbol.json